**IU000128 Coding3: Exploring Machine Intelligence Part 2** 

For more info, please check [Coding3 part 1](https://colab.research.google.com/drive/1NPteSsCJ89l697_ztAmcLCU_pGGjzvi5?usp=sharing)

--- 

## Char-RNN and Text Generation - by YIFAN FENG


#### Main Reference
*   [Char-RNN Blog](https://geeks-today.medium.com/rnn-character-level-text-generation-with-tensorflow-2-0-from-scratch-to-deep-insights-41bac0e07f86)
*   [Char-RNN Project: Trump-like Tweets](https://github.com/jctestud/char-rnn)

### Import Libs and Set up Env

In [ ]:
from google.colab import drive #Access GoogleDrive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np
import os
import time 

### Text Processing: from Word to Character

In [ ]:
#Import textfile generated from PART_ONE
#text = open('./oxford_ai_cleaned.txt', 'rb').read().decode(encoding='utf-8') 
text = open('/content/drive/MyDrive/Coding3/oxford_ai_update.txt', 'rb').read().decode(encoding='utf-8') 

In [ ]:
#Word Embedding

vocab = sorted(set(text)) 
#print('Length of text: {} characters.'.format(len(text)))
char2idx = {char:i for i, char in enumerate(vocab)}
idx2char = np.array(vocab)
text_encoded = [char2idx[c] for c in text]
text_encoded = np.array(text_encoded)

In [ ]:
seq_length = 100 #set the length sentence for one input 
example_per_epoch = len(text)//seq_length # one example of seq_length characters.

# Create training examples / targets 
char_dataset = tf.data.Dataset.from_tensor_slices(text_encoded) #Decompose to character level
sequences = char_dataset.batch(batch_size=seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text
dataset = sequences.map(split_input_target)
'''
for input_ex, target_ex in dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_ex.numpy()])))
    print('Output data:',repr(''.join(idx2char[target_ex.numpy()])))
'''

"\nfor input_ex, target_ex in dataset.take(1):\n    print('Input data: ', repr(''.join(idx2char[input_ex.numpy()])))\n    print('Output data:',repr(''.join(idx2char[target_ex.numpy()])))\n"

### Reconstruct Char-RNN Model 


*   [Char-RNN Source Code](https://github.com/sherjilozair/char-rnn-tensorflow)





In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


In [ ]:
#??dataset.shuffle

In [ ]:
vocab_size = len(vocab)
embedding_dim = 512 #optional: 256

def build_model(vocab_size, embedding_dim, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size,
                                  output_dim = embedding_dim,
                                  batch_input_shape = [batch_size, None]),

        tf.keras.layers.GRU(units = 64,
                            return_sequences= True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform',
                            dropout = 0.1), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.GRU(units = 128,
                            return_sequences= True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform',
                            dropout = 0.1), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.GRU(units = 256,
                            return_sequences= True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform',
                            dropout = 0.1), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),

         tf.keras.layers.GRU(units = 512,
                            return_sequences= True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform',
                            dropout = 0.1), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),


        tf.keras.layers.Dense(vocab_size)
    ])
    
    return model

In [ ]:
model = build_model(vocab_size = len(vocab),
                    embedding_dim = embedding_dim,
                    batch_size = BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 512)           179200    
                                                                 
 gru (GRU)                   (64, None, 64)            110976    
                                                                 
 batch_normalization (BatchN  (64, None, 64)           256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (64, None, 64)            0         
                                                                 
 gru_1 (GRU)                 (64, None, 128)           74496     
                                                                 
 batch_normalization_1 (Batc  (64, None, 128)          512       
 hNormalization)                                        

In [ ]:
# First check the shape of the output:
for input_ex_batch, target_ex_batch in dataset.take(1):
    ex_batch_prediction = model(input_ex_batch) # simply it takes input and calculate output with initial weights.
    print(ex_batch_prediction.shape, "# (batch_size, sequence_length, vocab_size)") 

(64, 100, 350) # (batch_size, sequence_length, vocab_size)


In [ ]:
sampled_indices = tf.random.categorical(ex_batch_prediction[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
#print(sampled_indices)

[ 97  58 277   1 175 111 290 304 121 274 235 175 205 220 312 236  65 219
 106 336 203   8  96 156 195  99  54 244  78 209 206 104 335  88 135 158
 186 223  31 284 120 260 207 123 267 301   8 103 209  55  18 298 116 132
  33 273  74  18 124 336 328 182 241 341 327 116 298  60  10  95  36 325
 309   3 105  84  19  28 344 324  50 205 174 120 323 122 332 269 263 143
  46 252 122 317 311 239 235 283 312 233]


In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
ex_batch_loss = loss(target_ex_batch, ex_batch_prediction)
#print("Prediction shape: ", ex_batch_prediction.shape, " # (batch_size, sequence_length, vocab_size)")
#print("Scaler loss: ", ex_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 350)  # (batch_size, sequence_length, vocab_size)
Scaler loss:  5.858283


In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'C:/Users/21036265/Desktop/charrnn_checkpoints'

# Checkpoint file only save training weight. Save after 10 epochs
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True, save_freq=10)

EPOCHS = 500 #early stop at 300 due to loss increase
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/500
290/290 [==============================] - 11s 37ms/step - loss: 2.7453
Epoch 2/500
290/290 [==============================] - 10s 36ms/step - loss: 1.8635
Epoch 3/500
290/290 [==============================] - 10s 36ms/step - loss: 1.7135
Epoch 4/500
290/290 [==============================] - 11s 36ms/step - loss: 1.6371
Epoch 5/500
290/290 [==============================] - 11s 37ms/step - loss: 1.5903
Epoch 6/500
290/290 [==============================] - 11s 36ms/step - loss: 1.5586
Epoch 7/500
290/290 [==============================] - 11s 36ms/step - loss: 1.5350
Epoch 8/500
290/290 [==============================] - 11s 36ms/step - loss: 1.5181
Epoch 9/500
290/290 [==============================] - 11s 36ms/step - loss: 1.5036
Epoch 10/500
290/290 [==============================] - 11s 36ms/step - loss: 1.4912
Epoch 11/500
290/290 [==============================] - 11s 37ms/step - loss: 1.4816
Epoch 12/500
290/290 [==============================] - 11s 37ms/step - lo

290/290 [==============================] - 11s 37ms/step - loss: 1.3433
Epoch 188/500
290/290 [==============================] - 11s 37ms/step - loss: 1.3436- ETA: 0s - loss: 1 - ETA: 0s - loss: 1.343
Epoch 189/500
290/290 [==============================] - 11s 37ms/step - loss: 1.3443 E
Epoch 190/500
290/290 [==============================] - 11s 36ms/step - loss: 1.3427
Epoch 191/500
290/290 [==============================] - 11s 37ms/step - loss: 1.3416
Epoch 192/500
290/290 [==============================] - 11s 37ms/step - loss: 1.3424
Epoch 193/500
290/290 [==============================] - 11s 37ms/step - loss: 1.3418
Epoch 194/500
290/290 [==============================] - 11s 37ms/step - loss: 1.3447
Epoch 195/500
290/290 [==============================] - 11s 39ms/step - loss: 1.3419
Epoch 196/500
290/290 [==============================] - 11s 37ms/step - loss: 1.3418
Epoch 197/500
290/290 [==============================] - 11s 37ms/step - loss: 1.3428
Epoch 198/500
290/290 [

KeyboardInterrupt: 

In [ ]:
#Restore training weights 
model = build_model(vocab_size, embedding_dim, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir), by_name=False ,skip_mismatch=False )

# Builds the model based on input shapes received.
model.build(tf.TensorShape([1, None])) 

### Generate Some Conditional Text!!!


In [ ]:
def generate_text(model, start_string, num_generate, temperature):
    
    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx [s] for s in start_string.lower()]
    # convert (x,y) shaped matrix to (1,x,y).
    input_eval = tf.expand_dims(input_eval, axis=0) 
    
    # Empty string to store our results
    text_generated = []
    
    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        
        # using a categorical distribution to predict the 
        # character returned by the model
        predictions = predictions / temperature
        
        # We got the predictions for every timestep but we 
        # want only last so first we take [-1] to consider on last 
        # predictions distribution only and after we try to get id 
        # from 1D array. Ex. we got '2' from a=['2'] by a[0].
        predicted_id = tf.random.categorical(predictions, 
                                             num_samples=1
                                            )[-1,0].numpy()
        
        # We pass the predicted character as the next input to the 
        # model along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        
        text_generated.append(idx2char[predicted_id])
        
    return (start_string + ''.join(text_generated))

In [ ]:
generate_text(model,start_string=u'Responsible AI is',num_generate=200,temperature=0.1)

'Responsible AI issues sections ai systems section algorith constion algorith conter algorithm simple algorithmation algorith develop alson conter artificial system accounted comple algorith algorither algorither ai sy'

In [ ]:
generate_text(model,start_string=u'Artificial Intelligence is',num_generate=200,temperature=0.2)

'Artificial Intelligence issuts procent proper comple stated comple ai sect ai see contion accounted algorithment proces social alson procention complecing ai system sections stantical interneted comple human procenting access '

In [ ]:
generate_text(model,start_string=u'Ethical AI is',num_generate=200,temperature=0.25)

'Ethical AI issibility scountificial conter accounted comple secter may interneter sections stantificial system human respons artificial setal result internations entificial ai ponted sect ai alson section proces c'

In [ ]:
generate_text(model,start_string=u'Accountability is',num_generate=200,temperature=0.15)

'Accountability issues ai algorither conteration ai social systems sections respect algorith algority comple state section conter process ai systems conseques ai proce conter artificial respect ai systems conseques acc'

In [ ]:
generate_text(model,start_string=u'Equitable AI is',num_generate=200,temperature=0.05)

'Equitable AI issues secte conter algorithm conter artificial systems sections sections ai systems section algorith constical ai systems sections sections proven ai stantificial section conter algorither algorither a'

In [ ]:
generate_text(model,start_string=u'Accountable AI is',num_generate=200,temperature=0.2)

'Accountable AI issess sections ai system constions designing advertion prove provers design contion algorith section experted ai accounted comple algorither sections experent algority sections sections ai sect ai cont'

In [ ]:
generate_text(model,start_string=u'Responsibility is',num_generate=200,temperature=0.15)

'Responsibility issues section algorithms secte algorithm conteration experting ai system ai sections stantern algority algorithm sections ai system constions sections ai systems sections accounted conter algorith cons'

In [ ]:
generate_text(model,start_string=u'Equity is',num_generate=200,temperature=0.05)

'Equity istion action algorith also contertions ai systems section conter algorith consition secte artificial interneted ai systems sections ai systems section algorith constion algorith constitute section ai s'

In [ ]:
generate_text(model,start_string=u'Ethics is',num_generate=200,temperature=0.15)

'Ethics issues sections ai systems comple algorithm sections ai stantificial intelligence sected contion algorith consticul antical respons artificial siment algorithm conter artificial systems conseques ai art'